In [11]:
import os
import numpy as np
from tqdm import tqdm as tqdm
import pandas as pd
import bz2
import subprocess
import pickle as pkl
from itertools import chain
from glob import glob
from collections import defaultdict
from IPython.display import HTML

In [12]:
working_path = '/cluster/raid/home/f80878961/beestrong/'
count_path = '{}data/allele_freqs/sync_files/'.format(working_path)
tlf_path = '{}tlf/'.format(working_path)
script_path = "/cluster/raid/home/f80878961/scripts/tmp_scripts/"

In [13]:
# bs_ids = pheno_df['Colony'].to_list()
# snp_nr = 7023976 # will need sanity check here

## Settings

In [14]:
snp_nr = 7023976 # wc -l BS17_0071.count -1

prefix = 'All'
if prefix == 'All':
    pheno_df = pd.read_table('/cluster/raid/home/f80878961/beestrong/data/metadata/whole_beestrong_data.csv').rename(
        columns={
            'num_bs':'Colony',
            'group':'Genetic group'
        })
else: 
    pheno_df = pd.read_csv('{}BeeStrong_metadata.csv'.format(tlf_path))

filtered_count_path = '{}data/allele_freqs/{}/'.format(working_path, prefix)
queen_geno_path = '{}data/queen_genotypes/{}/'.format(working_path, prefix)

bs_ids = sorted([x.split('/')[-1].split('.')[0] for x in glob('{}*.count.bz2'.format(count_path))])

chunk_size = 101 # 15 chunks
batch_file = '{}bsid_batch.txt'.format(filtered_count_path)

# bs_ids = pheno_df['Colony'].to_list()
print(len(set(bs_ids).intersection(pheno_df['Colony'].to_list())))
print(len(bs_ids))
assert pheno_df['Colony'].to_list() == bs_ids

# for SNP filtering
min_prev = 0.1 # prevalence threshold for common alleles (if >2 common allele, then real tri-allelic)
min_depth = 10 # both mean and median depth 
max_depth = 50

1513
1513


## Merge allele frequencies .sync files

### Write per-allele count matrices (SNP x colony for A, T, C, G)

In [23]:
def write_merge_sync_script(script_fn, array_str):
    runstr="""#!/bin/bash -l
#SBATCH --array=ARRAY_STRING
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --mem=20g
#SBATCH --time=4:00:00
#SBATCH --job-name=merge_sync
#SBATCH --output=%x_%A_%a.out
#SBATCH --error=%x_%A_%a.err

modulesld
ebld
module use /software/anaconda3/envs/eb/easybuild/modules/all
conda activate planb

# file listing batches of BeeStrong ids (bs_id) for parallel computing
batch_file=$1
snp_nr=$2
count_path=$3
out_path=$4

batch_id=$SLURM_ARRAY_TASK_ID

# getting bs_ids
bs_ids=$(sed -n ${batch_id}'{p;q}' ${batch_file})

python ~/scripts/plan-b-omics/bin/merge_sync.py --batch_id ${batch_id} --snp_nr ${snp_nr} --bs_ids ${bs_ids} --count_path ${count_path} --out_path ${out_path}

echo DONE""".replace("ARRAY_STRING", array_str)
    with open(script_fn, 'w') as outf:
        outf.write(runstr)

In [25]:
# create batch file for first time
with open(batch_file, "w") as outf:
    for i in range(0, len(bs_ids), chunk_size):
        chunk = bs_ids[i:i+chunk_size]
        outf.write(",".join(chunk) + "\n")

In [28]:
array_str = '1-15'
script_fn = '{}merge_sync.run'.format(script_path)
write_merge_sync_script(script_fn, array_str)

In [29]:
%%bash -s "$script_fn" "$batch_file" "$snp_nr" "$count_path" "$filtered_count_path"
cd $5
sbatch $1 $2 $3 $4 $5

Submitted batch job 1388883


### Merge into 3-D matrix

In [33]:
with open(batch_file, "r") as inf:
    batch_bs_ids = [x.rstrip().split(',') for x in inf.readlines()]
assert sum([x == bs_ids[i] for i, x in enumerate(list(chain(*batch_bs_ids)))]) == len(bs_ids)

In [34]:
%%time
allele2jj = {'A': 0, 'T': 1, 'C':2, 'G':3}
count_mat = np.zeros((snp_nr, len(bs_ids), len(allele2jj)), dtype=np.uint16)

for allele, jj in allele2jj.items():
    print(allele, jj)
    
    for j, b_bs_ids in enumerate(batch_bs_ids):
        batch_id = j + 1
        batch_size = len(b_bs_ids)
        with open('{}count_{}_{}.txt'.format(filtered_count_path, allele, batch_id), 'r') as inf:
            i = 0
            for l in tqdm(inf):                    
                count_mat[i, j * chunk_size : j * chunk_size + batch_size, jj] = np.array(l.split()[2:], dtype=np.int16)
                i += 1

A 0


7023976it [01:27, 79911.85it/s]
7023976it [01:12, 96871.43it/s]
7023976it [01:13, 95607.20it/s]
7023976it [01:12, 96395.80it/s]
7023976it [01:13, 95634.25it/s]
7023976it [01:13, 95841.95it/s]
7023976it [01:12, 96750.18it/s]
7023976it [01:13, 96216.97it/s]
7023976it [01:13, 96170.55it/s]
7023976it [01:13, 94955.64it/s]
7023976it [01:10, 99338.00it/s] 
7023976it [01:10, 100071.38it/s]
7023976it [01:10, 99098.12it/s] 
7023976it [01:20, 86892.53it/s]
7023976it [01:19, 88464.59it/s]


T 1


7023976it [01:20, 87638.04it/s]
7023976it [01:19, 88887.25it/s]
7023976it [01:20, 87229.69it/s]
7023976it [01:21, 86617.28it/s]
7023976it [01:21, 86346.12it/s]
7023976it [01:20, 87418.80it/s]
7023976it [01:20, 87219.67it/s]
7023976it [01:22, 85541.27it/s]
7023976it [01:22, 85528.61it/s]
7023976it [01:22, 85412.89it/s]
7023976it [01:21, 86602.88it/s]
7023976it [01:21, 86004.87it/s]
7023976it [01:20, 86980.82it/s]
7023976it [01:21, 86634.61it/s]
7023976it [01:19, 87840.42it/s]


C 2


7023976it [01:22, 85362.98it/s]
7023976it [01:21, 86695.99it/s]
7023976it [01:21, 86272.75it/s]
7023976it [01:21, 85767.50it/s]
7023976it [01:23, 83933.48it/s]
7023976it [01:21, 85708.65it/s]
7023976it [01:22, 85253.64it/s]
7023976it [01:22, 84884.01it/s]
7023976it [01:23, 84503.23it/s]
7023976it [01:23, 84575.33it/s]
7023976it [01:23, 84403.22it/s]
7023976it [01:22, 85003.84it/s]
7023976it [01:22, 84759.25it/s]
7023976it [01:23, 84321.06it/s]
7023976it [01:21, 86137.15it/s]


G 3


7023976it [01:21, 86597.14it/s]
7023976it [01:21, 86523.52it/s]
7023976it [01:23, 84421.79it/s]
7023976it [01:21, 85817.88it/s]
7023976it [01:22, 85413.62it/s]
7023976it [01:22, 85604.58it/s]
7023976it [01:22, 85174.04it/s]
7023976it [01:23, 84585.02it/s]
7023976it [01:24, 83101.66it/s]
7023976it [01:22, 84669.79it/s]
7023976it [01:22, 84762.85it/s]
7023976it [01:22, 85111.30it/s]
7023976it [01:21, 86124.85it/s]
7023976it [01:22, 85098.11it/s]
7023976it [01:20, 86877.37it/s]

CPU times: user 1h 9min 1s, sys: 10min 24s, total: 1h 19min 25s
Wall time: 1h 20min 4s


In [35]:
%%time
with open('{}count_mat.pkl'.format(filtered_count_path), 'wb') as outf:
    pkl.dump(count_mat, outf, protocol=pkl.HIGHEST_PROTOCOL)

CPU times: user 909 ms, sys: 2min 15s, total: 2min 16s
Wall time: 2min 33s


## Filter SNPs and recode alleles

filtering 
1. monoallelic filtering : 3/4 tables empty
2. mitochondrial SNP: contig NC_001566.1
3. tri-allelic SNP filtering: >2 allele present in min. 10% colonies, else delete variants from third and fourth alleles
4. min-max depth (10/50, on mean AND median) (after tri-allelic variants deletions, also not including N and deletions)

In [5]:
%%time
with open('{}count_mat.pkl'.format(filtered_count_path), 'rb') as inf:
    count_mat = pkl.load(inf)
print(count_mat.nbytes/1000000000)

85.018205504
CPU times: user 0 ns, sys: 1min 10s, total: 1min 10s
Wall time: 1min 26s


In [6]:
with open(batch_file, "r") as inf:
    batch_bs_ids = [x.rstrip().split(',') for x in inf.readlines()]
assert sum([x == bs_ids[i] for i, x in enumerate(list(chain(*batch_bs_ids)))]) == len(bs_ids)

In [7]:
batch_i = 0
batch_ii = len(bs_ids)
print(batch_ii)

1513


In [8]:
%%time
# counts the number of A, T, C, G allele per SNP # should take 7min
allele_prev = np.sum(count_mat[:, batch_i:batch_ii, :] > 0, axis=1)
allele_prev

CPU times: user 2min 44s, sys: 5.78 s, total: 2min 50s
Wall time: 2min 51s


array([[   1, 1513,    3,    9],
       [   4, 1513,   27,    2],
       [1513,   11,    2,   22],
       ...,
       [1437,  849,  118,  337],
       [ 397, 1364, 1058,  218],
       [ 335, 1504,  310,  170]])

In [9]:
%%time
# if count 3/4 alleles == 0, then monoallelic
mono_mask = np.sum(allele_prev > 0, axis=1) == 1
print(np.sum(mono_mask))

# tri-allelic stringent
tri_stringent_mask = np.sum(allele_prev > 0, axis=1) > 2
print(np.sum(tri_stringent_mask))

# tri-allelic relaxed (filter SNP only when >2 common alleles, i.e., an allele present in >= 10% of colonies)
tri_mask = np.sum(allele_prev >= ((batch_ii - batch_i) * min_prev), axis=1) > 2
print(np.sum(tri_mask))

3261
6963287
17207
CPU times: user 433 ms, sys: 41.7 ms, total: 475 ms
Wall time: 479 ms


In [10]:
# iterate through stringent tri-allelic SNPs: filter if ambigous else recode (delete 3rd and fourth)
ambi_mask = np.full(snp_nr, False)
for i in tqdm(np.argwhere(tri_stringent_mask).flatten()):

    # get number of alleles and sort by prevalence 
    atcg_prev = allele_prev[i, :]
    alleles_sorted_by_prevalence = np.argsort(atcg_prev)

    # if 2nd and 3rd alleles are equally common --> ambiguous --> filter
    if atcg_prev[alleles_sorted_by_prevalence[1]] == atcg_prev[alleles_sorted_by_prevalence[2]]:
        ambi_mask[i] = True

    # else delete 3rd and 4rth alleles
    else:
        count_mat[i, batch_i:batch_ii, alleles_sorted_by_prevalence[:2]] = 0

np.sum(ambi_mask)

100%|██████████| 6963287/6963287 [00:50<00:00, 138304.88it/s]


122115

In [11]:
%%time
# depth filtering --> if to, long I could consider iterating ...to have an overview of the time it takes # 8 min
depth  = np.sum(count_mat[:, batch_i:batch_ii, :], axis = 2)

CPU times: user 2min 23s, sys: 58 s, total: 3min 21s
Wall time: 3min 22s


In [12]:
%%time
mean_depth = np.mean(depth, axis = 1)
median_depth = np.full(snp_nr, -1) # np.median breaks the kernel somehow
for i in tqdm(range(snp_nr)):
    median_depth[i] =  np.median(depth[i])

depth_mask = (mean_depth < min_depth) | (mean_depth > max_depth) | (median_depth < min_depth) | (median_depth > max_depth) 
np.sum(depth_mask)

100%|██████████| 7023976/7023976 [02:47<00:00, 41860.60it/s]


CPU times: user 3min 4s, sys: 686 ms, total: 3min 5s
Wall time: 3min 7s


123823

In [13]:
%%time
chr_ids = [''] * snp_nr
snp_pos = [''] * snp_nr
mito_mask = np.full(snp_nr, False)
with open('{}count_A_1.txt'.format(filtered_count_path), 'r') as inf:
    i = 0
    for l in tqdm(inf):
        ch = l.split()[0]
        pos = l.split()[1]
        chr_ids[i] = ch
        snp_pos[i] = pos
        if ch == 'NC_001566.1':
            mito_mask[i] = True
        i += 1
np.sum(mito_mask)

7023976it [00:25, 272771.65it/s]

CPU times: user 23.4 s, sys: 2.03 s, total: 25.4 s
Wall time: 25.8 s


287

In [14]:
# combine masks
mask = mono_mask + tri_mask + depth_mask + mito_mask + ambi_mask
print(np.sum(mask))
print(snp_nr - np.sum(mask))

257718
6766258


### Write PLINK bim_like files

1. Chromosome code
2. Variant ID
3. Position in centimorgans (safe to use dummy value of '0')
4. Base-pair coordinate (1-based; limited to 231-2)
5. ALT ('A1' in PLINK 1.x) allele code
6. REF ('A2' in PLINK 1.x) allele code

In [15]:
# find reference and alternative allele per SNP (REF: the one with the most counts)
jj2allele = np.array(['A', 'T', 'C', 'G'])
ref_alt = np.full((snp_nr, 2), '.')
for i in tqdm(np.argwhere(~mask).flatten()):
    ref_alt[i] = jj2allele[np.argsort(np.sum(count_mat[i, :, :], axis=0))[::-1][:2]]

100%|██████████| 6766258/6766258 [03:38<00:00, 30928.81it/s]


In [22]:
bim_like = pd.DataFrame({
    'CHROM': np.array(chr_ids)[~mask],
    'SNP_ID': np.array(snp_pos)[~mask],
    'GEN_DIST': ['0'] * np.sum(~mask),
    'POS': np.array(snp_pos)[~mask],
    'ALT': ref_alt[~mask][:, 1],
    'REF': ref_alt[~mask][:, 0]
})

# recode chromosome and SNP identifiers into integers
chr2int = dict(zip(list(dict.fromkeys(bim_like['CHROM'])), range(1, 1 + len(set(bim_like['CHROM'])))))
bim_like['CHROM'] = bim_like['CHROM'].replace(chr2int)

bim_like['CHROM'] = bim_like['CHROM'].astype(str)
bim_like['GEN_DIST'] = bim_like['GEN_DIST'].astype(str)
bim_like['SNP_ID'] = bim_like[["CHROM", "POS"]].agg(":".join, axis=1)

/tmp/ipykernel_2652648/1469035007.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bim_like['CHROM'] = bim_like['CHROM'].replace(chr2int)


In [23]:
bim_like

,CHROM,SNP_ID,GEN_DIST,POS,ALT,REF
0,1,1:5671,0,5671,G,T
1,1,1:5698,0,5698,C,T
2,1,1:6621,0,6621,G,A
3,1,1:6717,0,6717,A,C
4,1,1:6823,0,6823,C,G
...,...,...,...,...,...,...
6766253,16,16:7225908,0,7225908,C,T
6766254,16,16:7225911,0,7225911,T,C
6766255,16,16:7226186,0,7226186,A,G
6766256,16,16:7226346,0,7226346,A,G


In [24]:
%%time
bim_like.to_csv('{}{}.bim_like'.format(queen_geno_path, prefix), header=False, index=False, sep='\t')

CPU times: user 6.17 s, sys: 395 ms, total: 6.57 s
Wall time: 6.73 s


### Export count_ref.txt and depth.txt files

In [ ]:
# TO DO: insert also alleles! --> aim to have same .bgs as Sonia!
# ,CHROM,POS,REF,ALT,BS16_0032...

In [25]:
%%time
# creating depth dataframe # 30min...
depth_df = pd.DataFrame(depth[~mask])

CPU times: user 8.46 s, sys: 15.3 s, total: 23.8 s
Wall time: 23.9 s


In [ ]:
depth_df.insert(0, 'CHROM',  bim_like['CHROM'])
depth_df.insert(1, 'POS', bim_like['POS'])
depth_df.insert(2, 'REF',  bim_like['REF'])
depth_df.insert(3, 'ALT', bim_like['ALT'])
depth_df.columns = ['CHROM', 'POS', 'REF', 'ALT'] + bs_ids[batch_i: batch_ii]

In [39]:
%%time
for gg in set(pheno_df['Genetic group']):
    print(gg)
    depth_df[['CHROM', 'POS', 'REF', 'ALT'] + list(pheno_df[pheno_df['Genetic group'] == gg]['Colony'])].to_csv('{}depth_{}.txt'.format(queen_geno_path, gg), header=True, index=False, sep='\t')

Caucasica
hybrid
Mellifera
Ligustica_Carnica
CPU times: user 23min 46s, sys: 2min 26s, total: 26min 13s
Wall time: 26min 31s


In [40]:
%%time
# getting the ref allele counts 5-8min...
count_ref  = np.max(count_mat[:, batch_i:batch_ii, :], axis = 2)

# creating reference allele count dataframe
count_ref_df = pd.DataFrame(count_ref[~mask])
count_ref_df.insert(0, 'CHROM',  bim_like['CHROM'])
count_ref_df.insert(1, 'POS', bim_like['POS'])
count_ref_df.insert(2, 'REF',  bim_like['REF'])
count_ref_df.insert(3, 'ALT', bim_like['ALT'])
count_ref_df.columns = ['CHROM', 'POS', 'REF', 'ALT'] + bs_ids[batch_i: batch_ii]

CPU times: user 4min 58s, sys: 4min 6s, total: 9min 4s
Wall time: 9min 5s


In [41]:
%%time
for gg in set(pheno_df['Genetic group']):
    print(gg)
    count_ref_df[['CHROM', 'POS', 'REF', 'ALT'] + list(pheno_df[pheno_df['Genetic group'] == gg]['Colony'])].to_csv('{}count_ref_{}.txt'.format(queen_geno_path, gg), header=True, index=False, sep='\t')

Caucasica
hybrid
Mellifera
Ligustica_Carnica
CPU times: user 22min 17s, sys: 1min 5s, total: 23min 22s
Wall time: 23min 39s


## Queen genotype reconstruction

In [42]:
def write_run_beethoven_script(script_fn):
    runstr="""#!/bin/bash -l
#SBATCH --cpus-per-task=10
#SBATCH --mem=40g
#SBATCH --time=48:00:00
#SBATCH --job-name=beethoven
#SBATCH --output=%x_%j.out
#SBATCH --error=%x_%j.err

modulesld
ebld
module use /software/anaconda3/envs/eb/easybuild/modules/all
conda activate planb

dir_in=$1
depth_file=$2
count_ref_file=$3
prefix=$4

genoqueen_hom ${dir_in} ${depth_file} ${count_ref_file} 4 10 ${prefix} 1000

echo DONE"""
    with open(script_fn, 'w') as outf:
        outf.write(runstr)

In [44]:
script_fn = '{}beethoven.run'.format(script_path)
write_run_beethoven_script(script_fn)

In [45]:
script_fn

'/cluster/raid/home/f80878961/scripts/tmp_scripts/beethoven.run'

In [46]:
gg = 'Caucasica'
command = ['sbatch', script_fn, queen_geno_path, 'depth_{}.txt'.format(gg), 'count_ref_{}.txt'.format(gg), '{}_{}'.format(prefix, gg)]
result = subprocess.run(command, cwd=queen_geno_path, capture_output=True, text=True)

In [47]:
for gg in set(pheno_df['Genetic group']):
    if gg == 'Caucasica':
        continue
    print(gg)
    command = ['sbatch', script_fn, queen_geno_path, 'depth_{}.txt'.format(gg), 'count_ref_{}.txt'.format(gg), '{}_{}'.format(prefix, gg)]
    result = subprocess.run(command, cwd=queen_geno_path, capture_output=True, text=True)

hybrid
Mellifera
Ligustica_Carnica


## Write PLINK .ped


### Split beethoven .prob files

In [160]:
prefix = 'queen'
plink_path = '{}data/queen_genotypes/hom_pop/'.format(working_path)
populations = ['Caucasica', 'Ligustica_Carnica', 'Mellifera', 'hybrid']
chunk_size = 25
prob_threshold = 0.95

In [153]:
# need to split .prob files into 3: AA, AR, RR --> easier to read probability of these genotypes simulateously then...
def split_prob_file(
    plink_path, prefix, pop):

    prob_file = '{}{}_{}.prob'.format(plink_path, prefix, pop)
    aa_file = '{}{}_{}_AA.prob'.format(plink_path, prefix, pop)
    ar_file = '{}{}_{}_AR.prob'.format(plink_path, prefix, pop)
    rr_file = '{}{}_{}_RR.prob'.format(plink_path, prefix, pop)
    
    out_aa = open(aa_file, 'w')
    out_ar = open(ar_file, 'w')
    out_rr = open(rr_file, 'w')
    
    with open(prob_file, 'r') as inf:
        headers = inf.readline()
        out_aa.write(headers.rstrip(',geno\n') + '\n')
        out_ar.write(headers.rstrip(',geno\n') + '\n')
        out_rr.write(headers.rstrip(',geno\n') + '\n')

        for l in tqdm(inf):
            split_line = l.rstrip().split(',')
            geno = split_line[-1]
            if geno == 'AA':
                out_aa.write(','.join(split_line[:-1]) + '\n')
            elif geno == 'AR':
                out_ar.write(','.join(split_line[:-1]) + '\n')
            elif geno == 'RR':
                out_rr.write(','.join(split_line[:-1]) + '\n')
            else: 
                print('GENO: {}'.format(geno))
                      
    out_aa.close()
    out_ar.close()
    out_rr.close()

In [125]:
for pop in populations:
    split_prob_file(plink_path, prefix, pop)

20293116it [00:46, 437729.20it/s]
20293116it [24:53, 13589.27it/s]
20293116it [15:02, 22474.99it/s]
20293116it [14:04, 24026.43it/s]


### Write .ped by population with probability threshold

debug

In [ ]:
def write_ped_woprob(
    plink_path, prefix, pop, chunk_bs_ids, batch_j, chunk_size):
    
    # index of column with first bs_id (skipping metadata columns)
    bgs_col_idx = 5
    
    bim_like = pd.read_csv('{}{}.bim_like'.format(plink_path, prefix), sep='\t', header=None)
    bgs_file = '{}{}_{}.bgs'.format(plink_path, prefix, pop)
    snp_nr = len(bim_like)
    
    start_j = batch_j * chunk_size
    ped_mat = np.full((len(chunk_bs_ids), 2 * snp_nr), '', dtype='U1')
    
    inf = open(bgs_file, 'r')

    # check we have the same bs ids in header than in batch
    header_bsids = inf.readline()
    assert header_bsids.rstrip().split(',')[bgs_col_idx:][start_j: start_j + len(chunk_bs_ids)] == chunk_bs_ids, 'bs_id mismatch'

    # iterate snps and colonies
    for i in tqdm(range(snp_nr)):
        ref_numbers = inf.readline().rstrip().split(',')[bgs_col_idx:]

        for j in range(start_j, start_j + len(chunk_bs_ids)):
            ref_nr = ref_numbers[j]

            # homozygote reference
            if (ref_nr == '2'):
                ped_mat[j - start_j, 2 * i] = bim_like.iloc[i, 5]
                ped_mat[j - start_j, 2 * i + 1] = bim_like.iloc[i, 5]
            # heterozygote
            elif (ref_nr == '1'):
                ped_mat[j - start_j, 2 * i] = bim_like.iloc[i, 5]
                ped_mat[j - start_j, 2 * i + 1] = bim_like.iloc[i, 4]
            # homozygote alternative
            elif (ref_nr == '0'):
                ped_mat[j - start_j, 2 * i] = bim_like.iloc[i, 4]
                ped_mat[j - start_j, 2 * i + 1] = bim_like.iloc[i, 4]
            # below prob_threshold
            elif ref_nr in {'0', '1', '2'}:
                ped_mat[j - start_j, 2 * i] = '0'
                ped_mat[j - start_j, 2 * i + 1] = '0'
            else: 
                print('ERROR')
            
    inf.close()

    ped_df = pd.DataFrame({
        'FID': ['0'] * len(chunk_bs_ids), 
        'IID': chunk_bs_ids,
        'PID': ['0'] * len(chunk_bs_ids),
        'MID': ['0'] * len(chunk_bs_ids),
        'SEX': ['0'] * len(chunk_bs_ids), 
        'PHENOTYPE': ['-9'] * len(chunk_bs_ids)
    })
    ped_df = ped_df.join(pd.DataFrame(ped_mat))
    ped_df.to_csv('{}{}_{}_{}.ped'.format(plink_path, prefix, pop, batch_j), header=False, index=False, sep='\t')

In [5]:

queen_geno_path = '{}data/queen_genotypes/{}/'.format(working_path, prefix)
plink_path = queen_geno_path
populations = ['Ligustica_Carnica', 'Mellifera']
populations = ['Ligustica_Carnica']
prob_threshold = 0


In [6]:
prefix = 'Sonia_All'
plink_path = '{}data/queen_genotypes/{}/'.format(working_path, prefix)
pop = 'Ligustica_Carnica'

In [7]:
batch_file = '{}bsid_batch_{}.txt'.format(plink_path, pop)
with open(batch_file, "r") as inf:
    batch_bs_ids = [x.rstrip().split(',') for x in inf.readlines()]
batch_file

'/cluster/raid/home/f80878961/beestrong/data/queen_genotypes/Sonia_All/bsid_batch_Ligustica_Carnica.txt'

In [8]:
batch_file

'/cluster/raid/home/f80878961/beestrong/data/queen_genotypes/Sonia_All/bsid_batch_Ligustica_Carnica.txt'

In [9]:
batch_j = 0
chunk_bs_ids = batch_bs_ids[batch_j]
chunk_size = 25

In [11]:
bim_like


,0,1,2,3,4,5
0,CHROM,SNP_ID,GEN_DIST,POS,ALT,REF
1,1,1:5671,0,5671,C,T
2,1,1:5698,0,5698,C,T
3,1,1:6621,0,6621,G,A
4,1,1:6717,0,6717,T,C
...,...,...,...,...,...,...
6831070,16,16:7225911,0,7225911,T,C
6831071,16,16:7225986,0,7225986,C,G
6831072,16,16:7226186,0,7226186,C,G
6831073,16,16:7226346,0,7226346,A,G


In [10]:
# index of column with first bs_id (skipping metadata columns)
bgs_col_idx = 5

bim_like = pd.read_csv('{}{}.bim_like'.format(plink_path, prefix), sep='\t', header=None)
bgs_file = '{}{}_{}.bgs'.format(plink_path, prefix, pop)
snp_nr = len(bim_like)

start_j = batch_j * chunk_size
ped_mat = np.full((len(chunk_bs_ids), 2 * snp_nr), '', dtype='U1')

inf = open(bgs_file, 'r')

# check we have the same bs ids in header than in batch
header_bsids = inf.readline()
assert header_bsids.rstrip().split(',')[bgs_col_idx:][start_j: start_j + len(chunk_bs_ids)] == chunk_bs_ids, 'bs_id mismatch'

/tmp/ipykernel_1824108/1242175706.py:4: DtypeWarning: Columns (0,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  bim_like = pd.read_csv('{}{}.bim_like'.format(plink_path, prefix), sep='\t', header=None)


In [ ]:
# iterate snps and colonies
for i in tqdm(range(snp_nr)):
    ref_numbers = inf.readline().rstrip().split(',')[bgs_col_idx:]

    for j in range(start_j, start_j + len(chunk_bs_ids)):
        ref_nr = ref_numbers[j]

        # homozygote reference
        if (ref_nr == '2'):
            ped_mat[j - start_j, 2 * i] = bim_like.iloc[i, 5]
            ped_mat[j - start_j, 2 * i + 1] = bim_like.iloc[i, 5]
        # heterozygote
        elif (ref_nr == '1'):
            ped_mat[j - start_j, 2 * i] = bim_like.iloc[i, 5]
            ped_mat[j - start_j, 2 * i + 1] = bim_like.iloc[i, 4]
        # homozygote alternative
        elif (ref_nr == '0'):
            ped_mat[j - start_j, 2 * i] = bim_like.iloc[i, 4]
            ped_mat[j - start_j, 2 * i + 1] = bim_like.iloc[i, 4]
        # below prob_threshold
        elif ref_nr in {'0', '1', '2'}:
            ped_mat[j - start_j, 2 * i] = '0'
            ped_mat[j - start_j, 2 * i + 1] = '0'
        else: 
            print('ERROR')
        
inf.close()

ped_df = pd.DataFrame({
    'FID': ['0'] * len(chunk_bs_ids), 
    'IID': chunk_bs_ids,
    'PID': ['0'] * len(chunk_bs_ids),
    'MID': ['0'] * len(chunk_bs_ids),
    'SEX': ['0'] * len(chunk_bs_ids), 
    'PHENOTYPE': ['-9'] * len(chunk_bs_ids)
})
ped_df = ped_df.join(pd.DataFrame(ped_mat))
ped_df.to_csv('{}{}_{}_{}.ped'.format(plink_path, prefix, pop, batch_j), header=False, index=False, sep='\t')

In [19]:
def write_write_ped_script(script_fn, array_str):
    runstr="""#!/bin/bash -l
#SBATCH --array=ARRAY_STRING
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --mem=50g
#SBATCH --time=6:00:00
#SBATCH --job-name=write_ped
#SBATCH --output=%x_%A_%a.out
#SBATCH --error=%x_%A_%a.err

modulesld
ebld
module use /software/anaconda3/envs/eb/easybuild/modules/all
conda activate planb

# file listing batches of BeeStrong ids (bs_id) for parallel computing
batch_file=$1
chunk_size=$2
plink_path=$3
prefix=$4
pop=$5
prob_threshold=$6
    
batch_id=$SLURM_ARRAY_TASK_ID

# getting bs_ids
chunk_bs_ids=$(sed -n ${batch_id}'{p;q}' ${batch_file})
    
python ~/scripts/plan-b-omics/bin/write_ped.py --chunk_bs_ids ${chunk_bs_ids} --batch_id ${batch_id} --chunk_size ${chunk_size} --plink_path ${plink_path} --prefix ${prefix} --pop ${pop} --prob_threshold ${prob_threshold}

echo DONE""".replace("ARRAY_STRING", array_str)
    with open(script_fn, 'w') as outf:
        outf.write(runstr)

In [46]:
queen_geno_path

'/cluster/raid/home/f80878961/beestrong/data/queen_genotypes/All/'

In [50]:
chunk_size = 25
prefix = 'All'
populations = ['Caucasica', 'Ligustica_Carnica', 'Mellifera', 'hybrid']

# if threshold = 0, then the .prob files are not used
prob_threshold = 0

In [51]:
# write batch files
for pop in populations:
    batch_file = '{}bsid_batch_{}.txt'.format(plink_path, pop)
    bs_ids = pheno_df[pheno_df['Genetic group'] == pop]['Colony'].to_list()
    print(len(bs_ids))
    
    batch_bs_ids = []
    with open(batch_file, "w") as outf:
        for i in range(0, len(bs_ids), chunk_size):
            chunk = bs_ids[i:i+chunk_size]
            batch_bs_ids.append(chunk)
            outf.write(",".join(chunk) + "\n")

703
407


In [12]:
### Sonia .bgs
def get_bs_ids_from_bgs(bgs_file, bgs_col_idx=5):
    with open(bgs_file, 'r') as inf:
        header_bsids = inf.readline()
    return header_bsids.rstrip().split(',')[bgs_col_idx:]

def write_bim_like_from_bgs(bgs_file, plink_path, prefix):
    '''
    basically takes the first few columns of the .bgs file to create a  bim like file (that I am using to compute ped files)
    '''
    bim_df = pd.read_csv(bgs_file, sep=',', usecols=range(5))
    bim_df['SNP_ID'] = bim_df['CHROM'].astype(str) + ':' + bim_df['POS'].astype(str)
    bim_df['GEN_DIST'] = ['0'] * len(bim_df)
    bim_df = bim_df[['CHROM', 'SNP_ID', 'GEN_DIST', 'POS', 'ALT', 'REF']]
    bim_df.to_csv('{}{}.bim_like'.format(plink_path, prefix), index=False, header=None, sep='\t')
    
chunk_size = 25

prefix = 'Sonia_All'
queen_geno_path = '{}data/queen_genotypes/{}/'.format(working_path, prefix)
plink_path = queen_geno_path
populations = ['Ligustica_Carnica', 'Mellifera']
populations = ['Ligustica_Carnica']
prob_threshold = 0

# write batch files (for Sonia)
for pop in populations:
    batch_file = '{}bsid_batch_{}.txt'.format(plink_path, pop)
    bs_ids = get_bs_ids_from_bgs('{}{}_{}.bgs'.format(plink_path, prefix, pop), bgs_col_idx=5)
    print(len(bs_ids))
    
    batch_bs_ids = []
    with open(batch_file, "w") as outf:
        for i in range(0, len(bs_ids), chunk_size):
            chunk = bs_ids[i:i+chunk_size]
            batch_bs_ids.append(chunk)
            outf.write(",".join(chunk) + "\n")

821


In [16]:
write_bim_like_from_bgs('/cluster/raid/home/f80878961/beestrong/data/queen_genotypes/Sonia_All/Sonia_All_Ligustica_Carnica.bgs', plink_path, prefix)

In [17]:
pop = 'Ligustica_Carnica'
# pop = 'Caucasica'
# pop = 'Mellifera'
# pop = 'hybrid'

batch_file = '{}bsid_batch_{}.txt'.format(plink_path, pop)
with open(batch_file, "r") as inf:
    batch_bs_ids = [x.rstrip().split(',') for x in inf.readlines()]
batch_file

'/cluster/raid/home/f80878961/beestrong/data/queen_genotypes/Sonia_All/bsid_batch_Ligustica_Carnica.txt'

In [20]:
array_str = '1-{}'.format(len(batch_bs_ids)) if len(batch_bs_ids) > 1 else '1'
script_fn = '{}write_ped.run'.format(script_path)
write_write_ped_script(script_fn, array_str)

In [21]:
%%bash -s "$script_fn" "$batch_file" "$chunk_size" "$plink_path" "$prefix" "$pop" "$prob_threshold" 
cd $4
sbatch $1 $2 $3 $4 $5 $6 $7

Submitted batch job 1390200


cat the ped files

### Write PLINK .map files

.map file is also needed to convert ped to bed (only map + bed)

CHR   SNP_ID   GEN_DIST   BP_POS

basically bim wihout REF and ALT

In [5]:
prefix = 'All'
prefix = 'Sonia_All'
queen_geno_path = '{}data/queen_genotypes/{}/'.format(working_path, prefix)
populations = ['Ligustica_Carnica']

In [6]:
bim_like = pd.read_csv('{}{}.bim_like'.format(queen_geno_path, prefix), sep='\t', header=None)

In [7]:
for pop in populations:
    print(pop)
    bim_like[[0, 1, 2, 3]].to_csv('{}{}_{}.map'.format(queen_geno_path, prefix, pop), header=False, index=False, sep='\t')

Ligustica_Carnica


convert to bed in CLI. Not sure how long it will take for convertion to bed...

ml PLINK

plink --file queen --make-bed --out queen --allow-extra-chr

## GREML H2 analyses

In [32]:
plink_path = '/cluster/raid/home/f80878961/beestrong/data/queen_genotypes/{}/'.format(plink_prefix)

plink_prefix = 'hom_pop'

plink_files = '{}{}'.format(plink_path, plink_prefix)

gcta_path = '/cluster/raid/home/f80878961/bin/gcta-1.95.0-linux-kernel-3-x86_64/gcta64' # cluster version fails for GRM 

greml_path = '/cluster/raid/home/f80878961/beestrong/data/heritability_greml/{}/'.format(plink_prefix)


lod = np.log(5e-06)

populations = ['Mellifera', 'Ligustica_Carnica', 'hybrid']

### Compute GRMs

In [50]:
plink_prefix = 'hom_pop'
plink_files = '{}{}'.format(plink_path, plink_prefix)

pop = 'all_pops'
maf = 0.01

grm_path = '{}{}/GRM_maf{}/'.format(greml_path, pop, str(maf).replace('.', ''))
if not os.path.exists(grm_path):
    os.makedirs(grm_path)
        
grm_files = '{}{}'.format(grm_path, plink_prefix)

In [31]:
%%bash -s "$grm_path" "$gcta_path" "$plink_files" "$maf" "$grm_files"
mkdir -p $1
cd $1
$2 --bfile $3 --make-grm --maf $4 --out $5

*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version v1.95.0 Linux
* Built at Jul 21 2025 17:30:18, by GCC 8.4
* (C) 2010-present, Yang Lab, Westlake University
* Please report bugs to Jian Yang <jian.yang@westlake.edu.cn>
*******************************************************************
Analysis started at 14:24:25 CET on Fri Jan 09 2026.
Hostname: node07

Options: 
 
--bfile /cluster/raid/home/f80878961/beestrong/data/queen_genotypes/hom_pop/hom_pop 
--make-grm 
--maf 0.05 
--out /cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/all_pops/GRM_maf005/hom_pop 

Note: GRM is computed using the SNPs on the autosomes.
Reading PLINK FAM file from [/cluster/raid/home/f80878961/beestrong/data/queen_genotypes/hom_pop/hom_pop.fam]...
1442 individuals to be included from FAM file.
1442 individuals to be included. 0 males, 0 females, 1442 unknown.
Reading PLINK BIM file from [/cluster/raid/home/f80878961/b

In [46]:
greml_path

'/cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/'

In [49]:
str(maf)

'0.05'

In [73]:
maf = 0.05
plink_prefix = 'queen'
for pop in populations:
    print(pop)
    plink_files = '{}{}_{}'.format(plink_path, plink_prefix, pop)
    
    grm_path = '{}{}/GRM_maf{}/'.format(greml_path, pop, str(maf).replace('.', ''))
    if not os.path.exists(grm_path):
        os.makedirs(grm_path)
    grm_files = '{}{}'.format(grm_path, plink_prefix)
    
    command = [gcta_path, '--bfile', plink_files, '--make-grm', '--maf' , str(maf), '--out', grm_files]
    result = subprocess.run(command, cwd=grm_path, capture_output=True, text=True)
    print(result.stderr)
    print(result.stdout) 

Mellifera

*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version v1.95.0 Linux
* Built at Jul 21 2025 17:30:18, by GCC 8.4
* (C) 2010-present, Yang Lab, Westlake University
* Please report bugs to Jian Yang <jian.yang@westlake.edu.cn>
*******************************************************************
Analysis started at 15:25:05 CET on Fri Jan 09 2026.
Hostname: node07

Options: 
 
--bfile /cluster/raid/home/f80878961/beestrong/data/queen_genotypes/hom_pop/queen_Mellifera 
--make-grm 
--maf 0.05 
--out /cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/Mellifera/GRM_maf005/queen 

Note: GRM is computed using the SNPs on the autosomes.
Reading PLINK FAM file from [/cluster/raid/home/f80878961/beestrong/data/queen_genotypes/hom_pop/queen_Mellifera.fam]...
405 individuals to be included from FAM file.
405 individuals to be included. 0 males, 0 females, 405 unknown.
Reading PLINK BIM file from [/cluste

Create input for GREML analyses
1. phenotype file .phen
2. categorical variables .covar
3. continuous variables .qcovar

In [75]:
def write_pheno_file(h2_path, h2_name, gcta_df, phenotype):
    fn = '{}{}.phen'.format(h2_path, h2_name)
    gcta_df[['0', 'Colony', phenotype]].fillna('NA').to_csv(fn, header=None, sep='\t', index=False)

def write_covar_file(h2_path, h2_name, gcta_df, fixed_covar, tax_covar):

    temp_gcta_df = gcta_df.copy()
    
    # replace abundance values by presence-absence for prevalence associations
    temp_gcta_df[temp_gcta_df[tax_covar] >= lod] = int(1)
    temp_gcta_df[temp_gcta_df[tax_covar] < lod] = int(0)

    fn = '{}{}.covar'.format(h2_path, h2_name)
    temp_gcta_df[['0', 'Colony'] + fixed_covar + tax_covar].to_csv(fn, header=None, sep='\t', index=False)

def write_qcovar_file(h2_path, h2_name, gcta_df, fixed_qcovar, tax_qcovar):
    fn = '{}{}.qcovar'.format(h2_path, h2_name)
    gcta_df[['0', 'Colony'] + fixed_qcovar + tax_qcovar].to_csv(fn, header=None, sep='\t', index=False)

Combine metadata and symbiosphere data

In [54]:
pheno_df = pd.read_csv('{}BeeStrong_metadata.csv'.format(tlf_path))
abund_df = pd.read_csv('{}Core_Family_LRA.csv'.format(tlf_path))
pca_df = pd.read_csv('{}PCA_Core_Family_LRA.csv'.format(tlf_path))

gcta_df = pheno_df.merge(abund_df, on='Colony').merge(pca_df.rename(columns={'Unnamed: 0': 'Colony'}), on='Colony')
gcta_df.insert(0, '0', [0] * len(gcta_df))

# reorder pheno df according to fam_df (order disrupted due to per-population genotype inference)
fam_df = pd.read_csv('{}hom_pop.fam'.format(plink_path), header=None, sep='\s') 
gcta_df = gcta_df.set_index("Colony").loc[fam_df[1].to_list()].reset_index()

# check consistent colony order between pheno df and plink
bs_ids = gcta_df['Colony'].to_list()
assert fam_df[1].to_list() == bs_ids, 'messed up colony order'

gcta_df

/tmp/ipykernel_2233136/1970459003.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  fam_df = pd.read_csv('{}hom_pop.fam'.format(plink_path), header=None, sep='\s')


,Colony,0,date,Genetic group,percent_Ligustica_Carnica,percent_Mellifera,percent_Caucasica,bee_weigth,nbr_pho_varroa,nbr_pho_varroa_100bee,...,PC58,PC59,PC60,PC61,PC62,PC63,PC64,PC65,PC66,PC67
0,BS16_0060,0,06/07/2016,Caucasica,0.118259,1.369552e-04,0.881604,51.0,0.0,0.0,...,-0.223545,0.611330,0.302651,0.131466,-0.060962,0.205536,-0.101299,-0.252870,0.160926,0.010749
1,BS16_0061,0,06/07/2016,Caucasica,0.109495,9.200239e-05,0.890413,52.0,1.0,0.0,...,1.083837,0.319977,-0.277766,0.184884,-0.267314,-0.058724,0.008251,-0.107998,-0.273400,0.016922
2,BS16_0162,0,31/08/2016,Caucasica,0.047143,1.376441e-05,0.952843,63.0,28.0,6.0,...,0.114503,-0.171535,-0.115155,0.327699,0.127601,-0.413191,0.045553,-0.106437,0.230031,0.392338
3,BS16_0163,0,31/08/2016,Caucasica,0.015421,1.019360e-05,0.984569,52.0,2.0,1.0,...,-0.256039,-0.478161,0.069915,0.570721,-0.081693,0.135746,0.021538,-0.012105,-0.083352,-0.242981
4,BS16_0164,0,31/08/2016,Caucasica,0.005619,4.687815e-08,0.994381,60.0,9.0,2.0,...,-0.114564,-0.172282,0.017520,0.432423,-0.139529,-0.082052,0.020244,-0.335893,0.113096,0.116470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,BS17_0506,0,01/08/2017,hybrid,0.418368,5.687723e-01,0.012860,47.5,1.0,0.0,...,-0.185675,-0.057137,-0.014365,0.275159,0.384569,-0.180188,0.010630,0.194236,0.054757,-0.261595
1438,BS17_0511,0,01/08/2017,hybrid,0.044593,7.929777e-01,0.162429,41.0,0.0,0.0,...,-0.049772,0.183526,-0.083507,-0.102922,-0.160323,0.213182,0.103019,-0.118458,-0.101629,-0.220722
1439,BS17_0512,0,01/08/2017,hybrid,0.113869,5.741510e-01,0.311980,39.2,0.0,0.0,...,0.277892,-0.524994,0.025260,0.413383,-0.185017,-0.411371,-0.207086,0.299036,0.171580,0.535910
1440,BS17_0513,0,01/08/2017,hybrid,0.554205,4.213804e-01,0.024415,43.0,0.0,0.0,...,0.616182,0.201392,-0.425561,0.144140,-0.430934,0.012346,-0.186331,0.307234,0.191650,0.425309


In [55]:
set(gcta_df['Genetic group'])

{'Caucasica', 'Ligustica_Carnica', 'Mellifera', 'hybrid'}

In [86]:
# maf = '001'
maf = '005'

pop = 'Mellifera'
pop = 'Ligustica_Carnica'
pop = 'hybrid'
# pop = 'all_pops'

if pop == 'all_pops':
    pop_gcta_df = gcta_df
    plink_prefix = 'hom_pop'
else:
    pop_gcta_df = gcta_df[gcta_df['Genetic group'] == pop]
    plink_prefix = 'queen'

default_covar = ['Apiary', 'Year']
default_qcovar = ['Day of Year']
vaf_prev = [
    'Oscillospiraceae',
    'Corynebacteriaceae',
    'Cannabaceae',
    'Aeromonadaceae',
    'Weeksellaceae',
    'Araliaceae',
    'Actinomycetaceae',
    'Trypanosomatidae',
    'Saccharomycodaceae', # v_pho specific
]
vaf_abund = [
    'Bifidobacteriaceae',
    'Lactobacillaceae',
    'Pseudomonadaceae',   
    'Debaryomycetaceae',
    'Bartonellaceae',  
    'Morganellaceae',
    'Rosaceae', # v_pho specific
    'Microbacteriaceae',  # v_pho specific
    'Enterobacteriaceae',  # v_pho specific
    'Nosematidae' # v_pho specific
]

top5vaf_prev = [
   'Oscillospiraceae',
    'Cannabaceae',
    'Trypanosomatidae'
]
top5vaf_abund = [
    'Bartonellaceae',  
    'Morganellaceae'
]

name2settings = {
    'default_v_meta': ('log_v_meta', default_covar, [], default_qcovar, []),
    'default_v_pho': ('log_v_pho', default_covar, [], default_qcovar, []),
    'default_DMR': ('DMR', default_covar, [], default_qcovar, []),
    'default_Recapping': ('Recapping', default_covar, [], default_qcovar, []),
    #'VAF_v_meta': ('log_v_meta', default_covar, vaf_prev, default_qcovar, vaf_abund),
    #'VAF_v_pho': ('log_v_pho', default_covar, vaf_prev, default_qcovar, vaf_abund),
    #'top5VAF_v_meta': ('log_v_meta', default_covar, top5vaf_prev, default_qcovar, top5vaf_abund),
    #'top5VAF_v_pho': ('log_v_pho', default_covar, top5vaf_prev, default_qcovar, top5vaf_abund),
    #'1PC_v_meta': ('log_v_meta', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 2)]),
    #'1PC_v_pho': ('log_v_pho', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 2)]),
    #'1PC_DMR': ('DMR', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 2)]),
    #'1PC_Recapping': ('Recapping', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 2)]),
    #'2PC_v_meta': ('log_v_meta', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 3)]),
    #'2PC_v_pho': ('log_v_pho', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 3)]),
    #'2PC_DMR': ('DMR', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 3)]),
    #'2PC_Recapping': ('Recapping', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 3)]),
    #'3PC_v_meta': ('log_v_meta', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 4)]),
    #'3PC_v_pho': ('log_v_pho', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 4)]),
    #'3PC_DMR': ('DMR', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 4)]),
    #'3PC_Recapping': ('Recapping', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 4)]),
    '4PC_v_meta': ('log_v_meta', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 5)]),
    '4PC_v_pho': ('log_v_pho', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 5)]),
    '4PC_DMR': ('DMR', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 5)]),
    '4PC_Recapping': ('Recapping', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 5)]),
    # '5PC_v_meta': ('log_v_meta', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 6)]),
    # '5PC_v_pho': ('log_v_pho', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 6)]),
    # '5PC_DMR': ('DMR', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 6)]),
    # '5PC_Recapping': ('Recapping', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 6)]),
    # '10PC_v_meta': ('log_v_meta', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 11)]),
    # '10PC_v_pho': ('log_v_pho', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 11)]),
    # '10PC_DMR': ('DMR', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 11)]),
    # '10PC_Recapping': ('Recapping', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 11)]),
    # '15PC_v_meta': ('log_v_meta', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 16)]),
    # '15PC_v_pho': ('log_v_pho', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 16)]),
    # '15PC_DMR': ('DMR', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 16)]),
    # '15PC_Recapping': ('Recapping', default_covar, [], default_qcovar, ['PC{}'.format(x) for x in range(1, 16)]),
    # 'PC5_v_meta': ('log_v_meta', default_covar, [], default_qcovar, ['PC5']), # 5, 1, 4, are the most associated with Varroa
    # 'PC5_v_pho': ('log_v_pho', default_covar, [], default_qcovar, ['PC5']),
    # 'PC145_v_meta': ('log_v_meta', default_covar, [], default_qcovar, ['PC1', 'PC4', 'PC5']),
    # 'PC145_v_pho': ('log_v_pho', default_covar, [], default_qcovar, ['PC1', 'PC4', 'PC5']),
}

In [87]:
grm_files

'/cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/Ligustica_Carnica/GRM_maf005/queen'

In [88]:
grm_path = '{}{}/GRM_maf{}/'.format(greml_path, pop, maf)
grm_files = '{}{}'.format(grm_path, plink_prefix)

for h2_name, (phenotype, fixed_covar, tax_covar, fixed_qcovar, tax_qcovar) in name2settings.items():
    print(h2_name)
    h2_path = '{}{}/{}/{}/'.format(greml_path, pop, maf, h2_name)

    if not os.path.exists(h2_path):
        os.makedirs(h2_path)
        
    write_pheno_file(h2_path, h2_name, pop_gcta_df, phenotype)
    write_covar_file(h2_path, h2_name, pop_gcta_df, fixed_covar, tax_covar)
    write_qcovar_file(h2_path, h2_name, pop_gcta_df, fixed_qcovar, tax_qcovar)

    command = [gcta_path, '--reml', '--grm',  grm_files , '--pheno', '{}{}.phen'.format(h2_path, h2_name), '--covar', '{}{}.covar'.format(h2_path, h2_name), 
           '--qcovar', '{}{}.qcovar'.format(h2_path, h2_name), '--out', '{}{}'.format(h2_path, h2_name)]
    result = subprocess.run(command, cwd=greml_path, capture_output=True, text=True)
    print(result.stderr)
    print(result.stdout)

default_v_meta

*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version v1.95.0 Linux
* Built at Jul 21 2025 17:30:18, by GCC 8.4
* (C) 2010-present, Yang Lab, Westlake University
* Please report bugs to Jian Yang <jian.yang@westlake.edu.cn>
*******************************************************************
Analysis started at 15:29:37 CET on Fri Jan 09 2026.
Hostname: node07

Accepted options:
--reml
--grm /cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/hybrid/GRM_maf005/queen
--pheno /cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/hybrid/005/default_v_meta/default_v_meta.phen
--covar /cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/hybrid/005/default_v_meta/default_v_meta.covar
--qcovar /cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/hybrid/005/default_v_meta/default_v_meta.qcovar
--out /cluster/raid/home/f80878961/beestrong/dat

In [94]:
maf = '001'
maf = '005'

pop = 'Mellifera'
# pop = 'Ligustica_Carnica'
#pop = 'hybrid'
# pop = 'all_pops'

phenotypes = ['v_meta', 'v_pho', 'Recapping', 'DMR']

# settings = ['default', 'VAF', 'top5VAF', '1PC', '2PC', '3PC', '4PC', '5PC', '10PC', '15PC', 'PC5', 'PC145']
settings = ['default', '4PC']

pheno2h2_list = defaultdict(list)
pheno2se_list = defaultdict(list)
pheno2pv_list = defaultdict(list)

for pheno in phenotypes:
    for sett in settings:
        h2_name = '{}_{}'.format(sett, pheno)
        h2_path = '{}{}/{}/{}/'.format(greml_path, pop, maf, h2_name)
        hsq_fn = '{}{}.hsq'.format(h2_path, h2_name)
        if not os.path.exists(hsq_fn): 
            h2 = np.nan
            se = np.nan
            pv = np.nan
        else:
            h2 = pd.read_table(hsq_fn).iloc[3, 1]
            se = pd.read_table(hsq_fn).iloc[3, 2]
            pv = pd.read_table(hsq_fn).iloc[8, 1]
            
        pheno2h2_list[pheno].append(h2)
        pheno2se_list[pheno].append(se)
        pheno2pv_list[pheno].append(pv)

In [95]:
h2_df = pd.DataFrame(pheno2h2_list)
h2_df.insert(0, 'setting', settings)
HTML(h2_df.round(3).to_html(index=False))

setting,v_meta,v_pho,Recapping,DMR
default,0.669,0.762,0.673,0.0
4PC,0.621,0.703,0.629,0.0


In [96]:
se_df = pd.DataFrame(pheno2se_list)
se_df.insert(0, 'setting', settings)
HTML(se_df.round(3).to_html(index=False))

setting,v_meta,v_pho,Recapping,DMR
default,0.159,0.150,0.165,0.132
4PC,0.170,0.164,0.174,0.138


In [44]:
pv_df = pd.DataFrame(pheno2pv_list)
pv_df.insert(0, 'setting', settings)
HTML(pv_df.to_html(index=False))

setting,v_meta,v_pho,Recapping,DMR
default,0.006356,0.000906,0.000117,0.33356
5PC,0.005243,0.001364,0.000022,0.30143


In [189]:
pv_df = pd.DataFrame(pheno2pv_list)
pv_df.insert(0, 'setting', settings)
pv_df

,setting,v_meta,v_pho,Recapping,DMR
0,default,0.000009,3.039900e-06,0.0,0.35711
1,VAF,0.000067,2.402300e-05,NA,NA
2,top5VAF,0.000021,2.408800e-06,NA,NA
3,5PC,0.000005,7.916900e-07,0.0,0.35697
4,10PC,0.000007,1.099400e-06,0.0,0.36479
5,15PC,0.000008,1.302400e-06,0.0,0.3471
6,PC5,0.000009,2.914100e-06,NA,NA
7,PC145,0.000008,1.559800e-06,NA,NA


In [ ]:
res_df = pd.DataFrame(pheno2h2_list)
res_df.insert(0, 'setting', settings)

In [171]:
pd.read_table(hsq_fn)

,Source,Variance,SE
0,V(G),0.000544,0.001827
1,V(e),0.036814,0.002078
2,Vp,0.037358,0.001625
3,V(G)/Vp,0.014569,0.048732
4,logL,1159.177000,NaN
5,logL0,1159.109000,NaN
6,LRT,0.134000,NaN
7,df,1.000000,NaN
8,Pval,0.357110,NaN
9,n,1280.000000,NaN


In [133]:
['PC{}'.format(x) for x in range(1, 6)]

['PC1', 'PC2', 'PC3', 'PC4', 'PC5']

In [110]:
phenotype = 'log_v_meta'
fixed_covar = ['Apiary', 'Year']
fixed_qcovar = ['Day of Year']
tax_covar = []
tax_qcovar = []
tax_covar = [
    'Oscillospiraceae',
    'Corynebacteriaceae',
    'Cannabaceae',
    'Aeromonadaceae',
    'Weeksellaceae',
    'Araliaceae',
    'Actinomycetaceae',
    'Trypanosomatidae',
    'Saccharomycodaceae', # v_pho specific
]
tax_qcovar = [
    'Bifidobacteriaceae',
    'Lactobacillaceae',
    'Pseudomonadaceae',   
    'Debaryomycetaceae',
    'Bartonellaceae',  
    'Morganellaceae',
    'Rosaceae', # v_pho specific
    'Microbacteriaceae',  # v_pho specific
    'Enterobacteriaceae',  # v_pho specific
    'Nosematidae' # v_pho specific
]

In [112]:
if not os.path.exists(h2_path):
    os.makedirs(h2_path)

write_pheno_file(gcta_df, h2_name, phenotype)
write_covar_file(gcta_df, h2_name, phenotype, fixed_covar, tax_covar)
write_qcovar_file(gcta_df, h2_name, phenotype, fixed_qcovar, tax_qcovar)

*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version v1.95.0 Linux
* Built at Jul 21 2025 17:30:18, by GCC 8.4
* (C) 2010-present, Yang Lab, Westlake University
* Please report bugs to Jian Yang <jian.yang@westlake.edu.cn>
*******************************************************************
Analysis started at 15:42:04 CET on Tue Jan 06 2026.
Hostname: node07

Accepted options:
--reml
--grm /cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/GRM/hom_pop
--pheno /cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/VAF/VAF.phen
--covar /cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/VAF/VAF.covar
--qcovar /cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/VAF/VAF.qcovar
--out /cluster/raid/home/f80878961/beestrong/data/heritability_greml/hom_pop/VAF/VAF

Note: This is a multi-thread program. You could specify the number of threads by the --th

In [ ]:
for gg in set(pheno_df['Genetic group']):
    command = ['sbatch', script_fn, dir_in, 'depth_{}.txt'.format(gg), 'count_ref_{}.txt'.format(gg), 'queen_{}'.format(gg)]
    result = subprocess.run(command, cwd=dir_in, capture_output=True, text=True)

In [199]:
columns = ['Colony', 'log_v_pho', 'log_v_meta', 'Apiary', 'date', 'Year', 'Day of Year']

# Varroa-Associated families
vaf_abund = [
    'Bifidobacteriaceae',
    'Lactobacillaceae',
    'Pseudomonadaceae',   
    'Debaryomycetaceae',
    'Bartonellaceae',  
    'Morganellaceae'
]
vaf_prev = [
    'Oscillospiraceae',
    'Corynebacteriaceae',
    'Cannabaceae',
    'Aeromonadaceae',
    'Weeksellaceae',
    'Araliaceae',
    'Actinomycetaceae',
    'Trypanosomatidae'
]

fn = '{}tlf/Core_Family_LRA.csv'.format(working_path)
abund_df = pd.read_csv(fn) #, index_col='Colony').reset_index

In [200]:
gcta_pheno_df = pd.DataFrame(
    {
        '0' : [0] * len(bs_ids),
        # '0': bs_ids,
        'Colony': bs_ids
    }
)
gcta_pheno_df = gcta_pheno_df.merge(pheno_df[columns], on='Colony', how='left')
gcta_pheno_df = gcta_pheno_df.merge(abund_df[['Colony'] + vaf_abund + vaf_prev], on='Colony', how='left')

# replace abundance values by presence-absence for prevalence associations
lod = np.log(5e-06)
gcta_pheno_df[gcta_pheno_df[vaf_prev] >= lod] = int(1)
gcta_pheno_df[gcta_pheno_df[vaf_prev] < lod] = int(0)

gcta_pheno_df = gcta_pheno_df.fillna('NA')

In [201]:
gcta_pheno_df

,0,Colony,log_v_pho,log_v_meta,Apiary,date,Year,Day of Year,Bifidobacteriaceae,Lactobacillaceae,...,Bartonellaceae,Morganellaceae,Oscillospiraceae,Corynebacteriaceae,Cannabaceae,Aeromonadaceae,Weeksellaceae,Araliaceae,Actinomycetaceae,Trypanosomatidae
0,0,BS16_0060,-1.987004,-15.168735,88.0,06/07/2016,2016,188,-6.146180,-4.997377,...,-6.744082,-9.018644,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
1,0,BS16_0061,-1.312186,-12.003170,88.0,06/07/2016,2016,188,-6.114353,-4.710165,...,-6.098915,-10.080948,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
2,0,BS16_0162,1.828127,-9.786422,88.0,31/08/2016,2016,244,-5.950136,-4.598848,...,-7.069882,-7.210088,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
3,0,BS16_0163,-0.619039,-15.168735,88.0,31/08/2016,2016,244,-5.707229,-4.439819,...,-6.104517,-10.880938,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
4,0,BS16_0164,0.741937,-10.537386,88.0,31/08/2016,2016,244,-5.405451,-4.150527,...,-7.050514,-9.954003,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,0,BS17_0506,-1.221672,-11.682297,128.0,01/08/2017,2017,213,-5.955760,-4.546061,...,-7.303314,-5.188458,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1438,0,BS17_0511,-1.987004,-14.661200,128.0,01/08/2017,2017,213,-5.556695,-3.947906,...,-5.801414,-8.585859,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
1439,0,BS17_0512,-1.987004,-15.168735,97.0,01/08/2017,2017,213,-5.081466,-4.056083,...,-9.312203,-9.878038,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1440,0,BS17_0513,-1.987004,-10.623075,97.0,01/08/2017,2017,213,-5.038656,-4.301295,...,-7.182466,-4.987978,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0


In [202]:
# phenotypes
fn = '{}{}.v_pho'.format(out_path, prefix)
gcta_pheno_df[['0', 'Colony', 'log_v_pho']].to_csv(fn, header=None, sep='\t', index=False)

fn = '{}{}.v_meta'.format(out_path, prefix)
gcta_pheno_df[['0', 'Colony', 'log_v_meta']].to_csv(fn, header=None, sep='\t', index=False)

pd.read_table(fn, header=None) 

,0,1,2
0,0,BS16_0060,-15.168735
1,0,BS16_0061,-12.003170
2,0,BS16_0162,-9.786422
3,0,BS16_0163,-15.168735
4,0,BS16_0164,-10.537386
...,...,...,...
1437,0,BS17_0506,-11.682297
1438,0,BS17_0511,-14.661200
1439,0,BS17_0512,-15.168735
1440,0,BS17_0513,-10.623075


In [203]:
# categorical covariable
fn = '{}{}.covar'.format(out_path, prefix)
gcta_pheno_df[['0', 'Colony', 'Apiary']].to_csv(fn, header=None, sep='\t', index=False)
# gcta_pheno_df[['0', 'Colony', 'Apiary', 'Year'] + vaf_prev].to_csv(fn, header=None, sep='\t', index=False)
# gcta_pheno_df[['0', 'Colony'] + vaf_prev].to_csv(fn, header=None, sep='\t', index=False)
# gcta_pheno_df[['0', 'Colony', 'Apiary']].to_csv(fn, header=None, sep='\t', index=False)
pd.read_table(fn, header=None)

,0,1,2
0,0,BS16_0060,88.0
1,0,BS16_0061,88.0
2,0,BS16_0162,88.0
3,0,BS16_0163,88.0
4,0,BS16_0164,88.0
...,...,...,...
1437,0,BS17_0506,128.0
1438,0,BS17_0511,128.0
1439,0,BS17_0512,97.0
1440,0,BS17_0513,97.0


In [204]:
# continuous covariable
fn = '{}{}.qcovar'.format(out_path, prefix)
gcta_pheno_df[['0', 'Colony', 'Day of Year']].to_csv(fn, header=None, sep='\t', index=False)
# gcta_pheno_df[['0', 'Colony', 'Day of Year'] + vaf_abund].to_csv(fn, header=None, sep='\t', index=False)
#gcta_pheno_df[['0', 'Colony'] + vaf_abund].to_csv(fn, header=None, sep='\t', index=False)
pd.read_table(fn, header=None)

,0,1,2
0,0,BS16_0060,188
1,0,BS16_0061,188
2,0,BS16_0162,244
3,0,BS16_0163,244
4,0,BS16_0164,244
...,...,...,...
1437,0,BS17_0506,213
1438,0,BS17_0511,213
1439,0,BS17_0512,213
1440,0,BS17_0513,213


# Old

### .bim_like

I need to create bim-like files but be careful they are overwritten with make-bed command

1. Chromosome code
2. Variant ID
3. Position in centimorgans (safe to use dummy value of '0')
4. Base-pair coordinate (1-based; limited to 231-2)
5. ALT ('A1' in PLINK 1.x) allele code
6. REF ('A2' in PLINK 1.x) allele code


In [111]:
%%time
with open('{}count_mat.pkl'.format(allele_count_path), 'rb') as inf:
    count_mat = pkl.load(inf)
print(count_mat.nbytes/1000000000)

81.028587136
CPU times: user 0 ns, sys: 1min 33s, total: 1min 33s
Wall time: 1min 52s


In [116]:
f_snp_nr = 6764372
snp_nr = 7023976
assert snp_nr == count_mat.shape[0]

In [117]:
%%time
# load original SNP ids
chr_ids = [''] * snp_nr
snp_pos = [''] * snp_nr
with open('{}data/gwas/BS17_0071.count'.format(working_path), 'r') as inf:
    print(inf.readline())
    i = 0
    for l in tqdm(inf):
        ch = l.split()[0]
        pos = l.split()[1]
        chr_ids[i] = ch
        snp_pos[i] = pos
        i += 1

CHROM POS REF ALT BS17-0071



7023976it [00:05, 1403874.48it/s]

CPU times: user 4.31 s, sys: 685 ms, total: 5 s
Wall time: 5.08 s


In [120]:
%%time
# load filtered SNPs
f_chr_ids = [''] * f_snp_nr
f_snp_pos = [''] * f_snp_nr
with open('{}filtered_snps.txt'.format(allele_count_path), 'r') as inf:
    inf.readline()
    i = 0
    for l in tqdm(inf):
        ch = l.split()[0]
        pos = l.split()[1]
        f_chr_ids[i] = ch
        f_snp_pos[i] = pos
        i += 1
assert f_snp_nr == i

6764372it [00:04, 1543827.37it/s]

CPU times: user 3.7 s, sys: 676 ms, total: 4.37 s
Wall time: 4.45 s


In [125]:
plink_path = '{}data/queen_genotypes/hom_pop/'.format(working_path)
bim = pd.read_csv('{}{}.bim'.format(plink_path, 'queen'), sep='\t', header=None)

In [132]:
# good
bim[3].to_list() == [int(x) for x in f_snp_pos]

True

In [133]:
# get back mask by comparing original snps and filtered snps. TO DO: if rerun filtering, include this part directly
snp_ids = ['{}_{}'.format(x, snp_pos[i]) for i, x in enumerate(chr_ids)]
f_snp_ids = ['{}_{}'.format(x, f_snp_pos[i]) for i, x in enumerate(f_chr_ids)]

assert len(snp_ids) == len(set(snp_ids))
assert len(f_snp_ids) == len(set(f_snp_ids))

snp_mask = ~np.isin(np.array(snp_ids), np.array(f_snp_ids))

assert np.sum(~snp_mask) == f_snp_nr

In [134]:
# find reference and alternative allele per SNP (REF: the one with the most counts. not sure if correct but probably REF and ALT are interchangeable) 
jj2allele = np.array(['A', 'T', 'C', 'G'])
ref_alt = np.full((snp_nr, 2), '.')
for i in tqdm(np.argwhere(~snp_mask).flatten()):
    ref_alt[i] = jj2allele[np.argsort(np.sum(count_mat[i, :, :], axis=0))[::-1][:2]]

100%|██████████| 6764372/6764372 [03:30<00:00, 32146.91it/s]


In [139]:
bim_like = pd.DataFrame({
    'CHR': f_chr_ids,
    'SNP_ID': f_snp_ids,
    'GEN_DIST': ['0'] * f_snp_nr,
    'BP_POS': f_snp_pos,
    'ALT': ref_alt[~snp_mask][:, 1],
    'REF': ref_alt[~snp_mask][:, 0]
})

In [142]:
# recode snp id
chr2int = dict(zip(list(dict.fromkeys(bim_like['CHR'])), range(1, 1 + len(set(bim_like['CHR'])))))
bim_like['CHR'] = bim_like['CHR'].replace(chr2int)

def recode_SNP_ids(old_id):
    chrom, pos = old_id.split('.1_')
    return '{}:{}'.format(chr2int['{}.1'.format(chrom)], pos)

bim_like['SNP_ID'] = bim_like['SNP_ID'].map(recode_SNP_ids)

/tmp/ipykernel_3354437/2140746017.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bim_like['CHR'] = bim_like['CHR'].replace(chr2int)


In [143]:
bim_like

,CHR,SNP_ID,GEN_DIST,BP_POS,ALT,REF
0,1,1:5671,0,5671,G,T
1,1,1:5698,0,5698,C,T
2,1,1:6621,0,6621,G,A
3,1,1:6717,0,6717,T,C
4,1,1:6823,0,6823,C,G
...,...,...,...,...,...,...
6764367,16,16:7225908,0,7225908,C,T
6764368,16,16:7225911,0,7225911,T,C
6764369,16,16:7226186,0,7226186,A,G
6764370,16,16:7226346,0,7226346,A,G


In [147]:
# good
bim[1].to_list() == bim_like['SNP_ID'].to_list()

True

In [149]:
%%time
bim_like.to_csv('{}queen.bim_like'.format(hom_pop_path), header=False, index=False, sep='\t')

CPU times: user 6.47 s, sys: 509 ms, total: 6.98 s
Wall time: 7.11 s


In [183]:
hom_pop_path

'/cluster/raid/home/f80878961/beestrong/data/queen_genotypes/hom_pop/'